# Scheduler

There are many situations where you may want to control the number of calcjobs that are running at the same time. For example:
- On the HPC cluster, user may has a limit on the maximum number of submissions that can be running at the same time.
- On the local workstation, user may want to limit the number of calcjobs that are running at the same time to avoid overloading the system.

## Managing the Scheduler

Start a scheduler with name `test-scheduler`:
```
workgraph scheduler start test-scheduler
```

Stop the scheduler:
```
workgraph scheduler stop test-scheduler
```

Show the status of the scheduler:
```
workgraph scheduler status test-scheduler
```

Show details of the processes submitted to the scheduler:
```
workgraph scheduler show test-scheduler
```

Set the maximum number of calcjobs that can be running at the same time:
```
workgraph scheduler set-max-jobs test-scheduler 5
```

Let's start a scheduler 

In [11]:
!workgraph scheduler start test-scheduler --max-calcjob 2 --max-process 10

Starting the scheduler ...


Check the status of the scheduler:

In [12]:
!workgraph scheduler status test-scheduler

Name                status      pk   waiting  running   calcjob  max_calcjob max_process
test-scheduler      Running  72507       0         0         0         2         10      



## Example Usage

Let's walk through an example where we creates four WorkGraphs with five calcjobs each.


In [13]:
from aiida_workgraph import WorkGraph
from aiida import load_profile, orm
from aiida.calculations.arithmetic.add import ArithmeticAddCalculation

load_profile()

# Use the calcjob: ArithmeticAddCalculation
code = orm.load_code("add@localhost")

for i in range(4):
    wg = WorkGraph("test_max_number_jobs")
    # Create N tasks
    for i in range(5):
        temp = wg.add_task(ArithmeticAddCalculation, name=f"add{i}", x=1, y=1, code=code)
        # Set a sleep option for each job (e.g., 10 seconds per job)
        temp.set({"metadata.options.sleep": 10})
    # submit the workgraph to a scheduler called "test-scheduler"
    wg.submit(scheduler="test-scheduler")

WorkGraph process created, PK: 72508
WorkGraph process created, PK: 72509
WorkGraph process created, PK: 72510
WorkGraph process created, PK: 72520


Note, all the WorkGraphs are submitted to a scheduler named `test-scheduler`. Now, you can check the progress of the Scheduler using the following command:



In [15]:
! workgraph scheduler show test-scheduler

Report: Scheduler: test-scheduler
   PK  Created    Process label                    Process State      Priorities
-----  ---------  -------------------------------  ---------------  ------------
72508  14s ago    WorkGraph<test_max_number_jobs>  ⏵ Waiting
72509  13s ago    WorkGraph<test_max_number_jobs>  ⏵ Waiting
72510  11s ago    WorkGraph<test_max_number_jobs>  ⏹ Created                  -2
72513  11s ago    ArithmeticAddCalculation         ⏵ Waiting
72516  10s ago    ArithmeticAddCalculation         ⏵ Waiting
72519  10s ago    ArithmeticAddCalculation         ⏹ Created                   0
72520  10s ago    WorkGraph<test_max_number_jobs>  ⏹ Created                   0
72523  10s ago    ArithmeticAddCalculation         ⏹ Created                  -3
72526  9s ago     ArithmeticAddCalculation         ⏹ Created                   0
72529  6s ago     ArithmeticAddCalculation         ⏹ Created                  -1
72532  6s ago     ArithmeticAddCalculation         ⏹ Created              


This command will display information about the currently running calcjobs, including the fact that there are a maximum of 2 calcjobs running simultaneously.

Additionally, you can monitor the progress visually by visiting `http://127.0.0.1:8000/workgraph`. Here, you can observe the duration of each task's execution over time.

Last but not least, one can stop and restart the scheduler at any time using the same name, all the information will be preserved.